In [1]:
import json
import requests
import mysql.connector
from mysql.connector import Error
import pandas as pd

In [2]:
stocks = ['GOOGL', 'META', 'AAPL', 'NFLX', 'AMZN']

product_data = {
    'url': "https://www.alphavantage.co/query?",
    'functions': ['TIME_SERIES_INTRADAY', 'TIME_SERIES_DAILY'],
    'intraday_interval': '15min',
    'output_size': 'compact',
    'products': ['Shopify', 'IBM'],
    'symbols': ['SHOP.TRT', 'IBM'],
    'APIKey': '7NVAEAZYAXPYKMZB'
}
print(product_data['url'])

In [3]:
#READ Data on Daily Time Series for Shopify - Symbol: SHOP.TRT - API-KEY: 7NVAEAZYAXPYKMZB
TSIurl = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=GOOGL&interval=30min&outputsize=full&apikey=7NVAEAZYAXPYKMZB'
response = requests.get(TSIurl)
timeseries_intraday = response.json()
StockURL = 'https://www.alphavantage.co/query?function=OVERVIEW&symbol=GOOGL&apikey=7NVAEAZYAXPYKMZB'
stocksdata = requests.get(StockURL).json()
#print(timeseries_intraday['Time Series (30min)'].keys())
#print(stocksdata['Name'])

#Create Database Connection
def server_connection(host_name, username, pwd):
    connection = None
    try:
        connection = mysql.connector.connect(host = host_name, user = username, passwd = pwd)
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

server = server_connection('localhost', 'root', '#Snroshan1998')

#Create a new database
def create_database(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        print("Database created successfully")
    except Error as err:
        print(f"Error: '{err}'")
        
create_database(server, "CREATE DATABASE SOEN6441")

In [4]:
def create_db_connection(host_name, username, pwd, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(host = host_name, user = username, passwd = pwd, database = db_name)
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

connection = create_db_connection('localhost', 'root', '#Snroshan1998', "SOEN6441")


MySQL Database connection successful


In [5]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query successful")
    except Error as err:
        print(f"Error: '{err}'")

In [6]:
create_table_stocks = """CREATE TABLE STOCKS (  STOCKNAME VARCHAR(1000),
                                                SYMBOL VARCHAR(20) PRIMARY KEY,
                                                SECTOR VARCHAR(1000),
                                                INDUSTRY VARCHAR(2000),
                                                MARKETCAP BIGINT,
                                                ASSETTYPE VARCHAR(300),
                                                COMPANY_DESCRIPTION VARCHAR(10000)
                                                )"""

execute_query(connection, create_table_stocks)

Error: '1050 (42S01): Table 'stocks' already exists'


#Execute a given Query
    
stockinfo = """INSERT INTO STOCKS VALUES ('%s', '%s', '%s', '%s', '%d', '%s', "%s")""" % (stocksdata['Name'], stocksdata['Symbol'], stocksdata['Sector'], stocksdata['Industry'], int(stocksdata['MarketCapitalization']), stocksdata['AssetType'], stocksdata['Description'])
#print(stockinfo)

execute_query(connection, stockinfo)


In [7]:
def add_stocks(stocks):
    for stock in stocks:
        url = 'https://www.alphavantage.co/query?function=OVERVIEW&symbol=%s&apikey=7NVAEAZYAXPYKMZB' % (stock)
        stock_data = requests.get(url).json()
        stock_query= """INSERT INTO STOCKS VALUES ('%s', '%s', '%s', '%s', '%d', '%s', "%s")""" % (stock_data['Name'], stock_data['Symbol'], stock_data['Sector'], stock_data['Industry'], int(stock_data['MarketCapitalization']), stock_data['AssetType'], stock_data['Description'])
        
        execute_query(connection, stock_query)

add_stocks(stocks)


Error: '1062 (23000): Duplicate entry 'GOOGL' for key 'stocks.PRIMARY''
Error: '1062 (23000): Duplicate entry 'META' for key 'stocks.PRIMARY''
Error: '1062 (23000): Duplicate entry 'AAPL' for key 'stocks.PRIMARY''
Error: '1062 (23000): Duplicate entry 'NFLX' for key 'stocks.PRIMARY''
Error: '1062 (23000): Duplicate entry 'AMZN' for key 'stocks.PRIMARY''


In [8]:
#ID INT AUTO_INCREMENT PRIMARY KEY, 
def create_stock_tables(stocks):
    for stock in stocks:
        create_table = """CREATE TABLE %s ( ID INT PRIMARY KEY,
                                            STOCKSYMBOL VARCHAR(20),
                                            DATENTIME DATETIME NOT NULL,
                                            OPEN FLOAT(9, 4),
                                            HIGH FLOAT(9, 4),
                                            LOW FLOAT(9, 4),
                                            CLOSE FLOAT(9, 4)
                                            )""" % (stock)

        execute_query(connection, create_table)

        
create_stock_tables(stocks)

Error: '1050 (42S01): Table 'googl' already exists'
Error: '1050 (42S01): Table 'meta' already exists'
Error: '1050 (42S01): Table 'aapl' already exists'
Error: '1050 (42S01): Table 'nflx' already exists'
Error: '1050 (42S01): Table 'amzn' already exists'


In [9]:
def get_values(stock, interval, data):
    intraday_values = []
    for datetime in data:
        intraday_values.append([datetime, float(data[datetime]['1. open']), float(data[datetime]['2. high']), float(data[datetime]['3. low']), float(data[datetime]['4. close'])])
    return (intraday_values)

In [10]:
def update_intraday_values(stocks, interval = '15min'):
    for stock in stocks:
        execute_query(connection, """DELETE FROM %s""" %(stock))
        url = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&interval=%s&symbol=%s&outputsize=full&apikey=7NVAEAZYAXPYKMZB' %(interval, stock)
        intraday_data = requests.get(url).json()
        values = get_values(stock, interval, intraday_data['Time Series (%s)' %(interval)])
        for n, i in zip(range(1, len(values) + 1), values):
            intraday_update_query= """INSERT INTO %s VALUES (%s, '%s', '%s', %f, %f, %f, %f)""" % (stock, n, intraday_data['Meta Data']['2. Symbol'], i[0], i[1], i[2], i[3], i[4])
            #print (intraday_update_query)
            execute_query(connection, intraday_update_query)
        #supply values into the query update above and pass into function to execute query
        
                                                                                           
update_intraday_values(stocks)

Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successf

Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successf

Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successf

Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successf

Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successf

...

Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successf

Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successf

Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successf

Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successf

Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successful
Query successf